# Homicidios intencionales en Ecuador

This notebook includes cleaning of murder data from Ecuador from Ecuador since 2014 to may 2025, and preparation for QGIS. Original data downloaded from [here](https://datosabiertos.gob.ec/dataset/?organization=ministerio-del-interior).

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
os.makedirs("data/clean_data/", exist_ok=True)

## Murders in 2025

In [4]:
df2025 = pd.read_excel("data/MDI_data/mdi_homicidios_intencionales_2025_ene_a_may.xlsx", sheet_name="1", skiprows=1, dtype=str)

In [5]:
df2025 = df2025.drop(columns={"Unnamed: 0"})

In [6]:
df2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4051 entries, 0 to 4050
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   tipo_muerte               4051 non-null   object
 1   zona                      4051 non-null   object
 2   subzona                   4051 non-null   object
 3   distrito                  4051 non-null   object
 4   circuito                  4051 non-null   object
 5   codigo_subcircuito        4051 non-null   object
 6   subcircuito               4051 non-null   object
 7   provincia                 4051 non-null   object
 8   codigo_provincia          4051 non-null   object
 9   canton                    4051 non-null   object
 10  codigo_canton             4051 non-null   object
 11  coordenada_y              4051 non-null   object
 12  coordenada_x              4051 non-null   object
 13  area_hecho                4051 non-null   object
 14  lugar                   

In [7]:
df2025["coordenada_x"] = df2025["coordenada_x"].str.replace(",", ".")
df2025["coordenada_y"] = df2025["coordenada_y"].str.replace(",", ".")

In [8]:
df2025.iloc[417][["coordenada_y", "coordenada_x"]]

coordenada_y      -1.6485
coordenada_x    -79.81508
Name: 417, dtype: object

In [9]:
decimal_pattern = r'^-?\d+(\.\d+)?$'

lat_valid = df2025["coordenada_x"].str.match(decimal_pattern)
lon_valid = df2025["coordenada_y"].str.match(decimal_pattern)

print("Valid latitudes:", lat_valid.sum(), "/", len(df2025))
print("Valid longitudes:", lon_valid.sum(), "/", len(df2025))
print("Invalid rows:\n", df2025[~(lat_valid & lon_valid)])

Valid latitudes: 4051 / 4051
Valid longitudes: 4051 / 4051
Invalid rows:
 Empty DataFrame
Columns: [tipo_muerte, zona, subzona, distrito, circuito, codigo_subcircuito, subcircuito, provincia, codigo_provincia, canton, codigo_canton, coordenada_y, coordenada_x, area_hecho, lugar, tipo_lugar, fecha_infraccion, hora_infraccion, arma, tipo_arma, presunta_motivacion, presun_motiva_observada, probable_causa_motivada, edad, medida_edad, sexo, genero, etnia, estado_civil, nacionalidad, discapacidad, profesion_registro_civil, instruccion]
Index: []

[0 rows x 33 columns]


In [10]:
lat_float = df2025["coordenada_y"].astype(float)
lon_float = df2025["coordenada_x"].astype(float)

lat_min, lat_max = -5.1, 2.0
lon_min, lon_max = -92.1, -75.0

in_lat_range = lat_float.between(lat_min, lat_max)
in_lon_range = lon_float.between(lon_min, lon_max)

valid_coords = in_lat_range & in_lon_range

all_valid = valid_coords.all()
print("All coordinates valid:", all_valid)
print("Invalid rows:", (~valid_coords).sum())

All coordinates valid: True
Invalid rows: 0


In [12]:
df2025.to_csv("data/clean_data/clean_2025_may.csv", index=False)

## Historical data on murders

In [13]:
df_historical = pd.read_excel("data/MDI_data/mdi_homicidios_intencionales_pm_2014_2024.xlsx", sheet_name="1", skiprows=1, dtype=str)

In [14]:
df_historical = df_historical.drop(columns={"Unnamed: 0"})

In [15]:
df_historical.info() #this df has one more column in comparison with the previous one

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30511 entries, 0 to 30510
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   tipo_muerte               30511 non-null  object
 1   zona                      30511 non-null  object
 2   subzona                   30511 non-null  object
 3   distrito                  30511 non-null  object
 4   circuito                  30511 non-null  object
 5   codigo_subcircuito        30511 non-null  object
 6   subcircuito               30511 non-null  object
 7   provincia                 30511 non-null  object
 8   codigo_provincia          30511 non-null  object
 9   canton                    30511 non-null  object
 10  codigo_canton             30511 non-null  object
 11  coordenada_y              30511 non-null  object
 12  coordenada_x              30511 non-null  object
 13  area_hecho                30511 non-null  object
 14  lugar                 

In [16]:
df_historical.head()

,tipo_muerte,zona,subzona,distrito,circuito,codigo_subcircuito,subcircuito,provincia,codigo_provincia,canton,...,medida_edad,sexo,genero,etnia,estado_civil,nacionalidad,discapacidad,profesion_registro_civil,instruccion,antecedentes
0,ASESINATO,ZONA 5,SANTA ELENA,LIBERTAD SALINAS,SALINAS CENTRO,24D02C05S02,SALINAS CENTRO 2,SANTA ELENA,24,SALINAS,...,A,HOMBRE,SIN_DATO,MESTIZO/A,CASADO,ECUADOR,NINGUNA,PINTOR,BACHILLERATO,NO
1,ASESINATO,ZONA 5,SANTA ELENA,LIBERTAD SALINAS,SALINAS CENTRO,24D02C05S02,SALINAS CENTRO 2,SANTA ELENA,24,SALINAS,...,A,HOMBRE,SIN_DATO,MESTIZO/A,SOLTERO,ECUADOR,NINGUNA,JORNALERO,BASICA,SIN_DATO
2,ASESINATO,ZONA 5,SANTA ELENA,LIBERTAD SALINAS,SALINAS CENTRO,24D02C05S02,SALINAS CENTRO 2,SANTA ELENA,24,SALINAS,...,A,HOMBRE,SIN_DATO,MESTIZO/A,SOLTERO,ECUADOR,NO DETERMINADO,COMERCIANTE,BACHILLERATO,SIN_DATO
3,ASESINATO,ZONA 5,SANTA ELENA,LIBERTAD SALINAS,SANTA ROSA,24D02C06S03,SANTA ROSA 3,SANTA ELENA,24,SALINAS,...,A,HOMBRE,SIN_DATO,MESTIZO/A,SOLTERO,ECUADOR,NINGUNA,ESTUDIANTE,BACHILLERATO,SIN_DATO
4,HOMICIDIO,ZONA 5,SANTA ELENA,LIBERTAD SALINAS,SANTA ROSA,24D02C06S01,SANTA ROSA 1,SANTA ELENA,24,SALINAS,...,A,HOMBRE,SIN_DATO,MESTIZO/A,SOLTERO,ECUADOR,NINGUNA,MAQUINISTA,BASICA,SIN_DATO


In [17]:
df_historical["fecha_infraccion"] = pd.to_datetime(df_historical["fecha_infraccion"], errors="coerce")

In [18]:
df_2324 = df_historical[df_historical["fecha_infraccion"].dt.year.isin([2023, 2024])]

In [19]:
len(df_2324)

15284

In [20]:
sample = df_2324[["coordenada_x", "coordenada_y"]].sample(20, random_state=1)
sample

,coordenada_x,coordenada_y
10715,-79.92582,-2.20022
19284,-7688714,0.09137
9580,-7995338,-325979
8998,-79.96852,-2.10025
15432,-79.58823,-2.14853
10676,-7992643,-221563
19949,-799491,-211454
10965,-79.92073,-2.23788
12153,-7988848,-225459
16739,-7946329,-217908


In [21]:
#Check if the missing decimal separator is the problem
y_sin_puntos = len(df_2324[~df_2324["coordenada_y"].str.contains(r"\.")])
y_puntos = len(df_2324[df_2324["coordenada_y"].str.contains(r"\.")]["coordenada_y"])
x_sin_puntos = len(df_2324[~df_2324["coordenada_x"].str.contains(r"\.")]["coordenada_x"])
x_puntos = len(df_2324[df_2324["coordenada_x"].str.contains(r"\.")]["coordenada_x"])

print(f"En x, hay {x_sin_puntos} valores sin puntos y {x_puntos} valores con puntos. Suma = {x_sin_puntos+x_puntos}")
print(f"En y, hay {y_sin_puntos} valores sin puntos y {y_puntos} valores con puntos. Suma = {y_sin_puntos+y_puntos}")
print(f"El len de la df es {len(df_2324)}")

En x, hay 7028 valores sin puntos y 8256 valores con puntos. Suma = 15284
En y, hay 5419 valores sin puntos y 9865 valores con puntos. Suma = 15284
El len de la df es 15284


In [22]:
lat_float = df_2324["coordenada_y"].astype(float)
lon_float = df_2324["coordenada_x"].astype(float)

lat_min, lat_max = -5.1, 2.0
lon_min, lon_max = -92.1, -75.0

in_lat_range = lat_float.between(lat_min, lat_max)
in_lon_range = lon_float.between(lon_min, lon_max)

valid_coords = in_lat_range & in_lon_range

all_valid = valid_coords.all()
print("All coordinates valid:", all_valid)
print("Invalid rows:", (~valid_coords).sum())

All coordinates valid: False
Invalid rows: 7036


In [23]:
def fix_lat(s):
    s = str(s).strip()
    if "." in s:
        return s
    sign = ''
    if s.startswith('-'):
        sign = '-'
        s = s[1:]
    if len(s) == 0:
        return s
    return sign + s[0] + '.' + s[1:]

mask = ~df_2324["coordenada_y"].astype(str).str.contains(r"\.", na=False)
df_2324.loc[mask, "coordenada_y"] = df_2324.loc[mask, "coordenada_y"].apply(fix_lat)

In [24]:
def fix_lon(s):
    if pd.isna(s):
        return s
    s = str(s).strip()
    if "." in s:
        return s
    s = s.lstrip("+-")  # Remove any sign
    if len(s) < 3:
        return None
    return '-' + s[:2] + '.' + s[2:]

mask = ~df_2324["coordenada_x"].astype(str).str.contains(r"\.", na=False)
df_2324.loc[mask, "coordenada_x"] = df_2324.loc[mask, "coordenada_x"].apply(fix_lon)

In [25]:
lat_float = df_2324["coordenada_y"].astype(float)
lon_float = df_2324["coordenada_x"].astype(float)

lat_min, lat_max = -5.1, 2.0
lon_min, lon_max = -92.1, -75.0

in_lat_range = lat_float.between(lat_min, lat_max)
in_lon_range = lon_float.between(lon_min, lon_max)

valid_coords = in_lat_range & in_lon_range

all_valid = valid_coords.all()
print("All coordinates valid:", all_valid)
print("Invalid rows:", (~valid_coords).sum())

All coordinates valid: False
Invalid rows: 1


In [26]:
df_2324[~df_2324["coordenada_x"].astype(float).between(lon_min, lon_max)][["coordenada_x", "coordenada_y"]]

,coordenada_x,coordenada_y
21156,None,0.


In [27]:
row_idx = 21156
df_2324.loc[row_idx, "coordenada_x"] = np.nan
df_2324.loc[row_idx, "coordenada_y"] = np.nan

In [28]:
df_2324[df_2324["coordenada_y"].str.contains("e", na=False)][["coordenada_x", "coordenada_y"]]

,coordenada_x,coordenada_y
20725,-79.3854,-4.e-04


In [29]:
row_idx = 20725
df_2324.loc[row_idx, "coordenada_y"] = "-0.0004"

In [30]:
df_2324.to_csv("data/clean_data/clean_2023-2024.csv", index=False)

## Merge both dfs

In [29]:
df_combined = pd.concat([df_2324, df2025], ignore_index=True)

In [30]:
lat_float = df_combined["coordenada_y"].astype(float)
lon_float = df_combined["coordenada_x"].astype(float)

lat_min, lat_max = -5.1, 2.0
lon_min, lon_max = -92.1, -75.0

in_lat_range = lat_float.between(lat_min, lat_max)
in_lon_range = lon_float.between(lon_min, lon_max)

valid_coords = in_lat_range & in_lon_range

all_valid = valid_coords.all()
print("All coordinates valid:", all_valid)
print("Invalid rows:", (~valid_coords).sum())

All coordinates valid: False
Invalid rows: 1


In [31]:
# Just a quick check of coordinates
sample = df_combined[["coordenada_x", "coordenada_y"]].sample(20, random_state=1)
sample

,coordenada_x,coordenada_y
6595,-79.89138,-2.24728
2563,-80.05879,0.06882
343,-80.84158,-0.97884
3894,-79.96048,-2.09608
18062,-79.5331,-1.80753
17608,-79.69679,-3.07801
15213,-79.653,0.97058
12268,-79.18938,-0.24421
15992,-80.70564,-0.95562
9316,-79.65262,0.97263


In [32]:
df_combined.loc[15367]

tipo_muerte                                ASESINATO
zona                                          ZONA 8
subzona                               D.M. GUAYAQUIL
distrito                            NUEVA PROSPERINA
circuito                            NUEVA PROSPERINA
codigo_subcircuito                       09D08C01S02
subcircuito                       NUEVA PROSPERINA 2
provincia                                     GUAYAS
codigo_provincia                                  09
canton                                     GUAYAQUIL
codigo_canton                                   0901
coordenada_y                                 -2.1224
coordenada_x                               -79.95852
area_hecho                                    URBANO
lugar                                    VIA PUBLICA
tipo_lugar                                   PUBLICO
fecha_infraccion                 2025-01-05 00:00:00
hora_infraccion                             09:00:00
arma                                   ARMA DE

In [33]:
df_combined.to_csv("clean_data/complete_data.csv", index=False)